In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
drone_path = '../../second.wav'
background_path = '../../test.wav'

drone_files = glob.glob(drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
drone_raw = load(drone_files)
background_raw = load(background_files)

(40960,)
(440320,)


# Data Processing

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_drone, y_drone = mfcc4(drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(mfcc_drone.shape, y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(40960,)
small end: (16, 9)
(440320,)
small end: (16, 13)
small end: (16, 5)
(9, 16, 16) (9,)
(106, 16, 16) (106,)


In [8]:
X = np.concatenate((mfcc_drone, mfcc_background), axis=0)
y = np.hstack((y_drone, y_background))
print(X.shape, y.shape)

(115, 16, 16) (115,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(115, 2)


In [10]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape, X_test.shape)
#print(X_val.shape, y_val.shape)
print(y_train.shape, y_test.shape)

(92, 16, 16) (23, 16, 16)
(92, 2) (23, 2)


In [12]:
# Save Data
np.save('../../model/X_train', X_train)
np.save('../../model/X_test', X_test)
#np.save('../../model/X_val', X_val)
#np.save('../../model/y_val', y_val)
np.save('../../model/y_train', y_train)
np.save('../../model/y_test', y_test)

### Until this part

In [13]:
# Load Data
X_train = np.load('../../model/X_train.npy')
X_test = np.load('../../model/X_test.npy')
#X_val = np.load('../../model/X_val.npy')
#y_val = np.load('../../model/y_val.npy')
y_train = np.load('../../model/y_train.npy')
y_test = np.load('../../model/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout

##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [14]:
tf.reset_default_graph()

In [15]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

learning_rate = 0.0002  # 0.005
training_epochs = 500 # 수정해봐

# Layer

In [16]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
# dropout넣어야하나
conv2 = tf.layers.conv2d(inputs=pool1, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=1)
# 여기도
flat = tf.reshape(pool2, [-1, 16*16*1])
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense2, units=2)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
Y_pred = tf.contrib.layers.fully_connected(logits,n_classes,activation_fn = None)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
#X_val2 = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [21]:
# model save
model_path = '../../model/CNN/cnn_model'
saver = tf.train.Saver()

# Trainning

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools as it

In [24]:
###########################
batch_size = 128 ## 조절해보자!
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):#training epoch 500 / batch_size 128 --> acc 90%
    avg_cost = 0
    val_avg_cost =0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch 
    
    #y_pred = sess.run(logits, feed_dict={X:X_val2})
    #y_pred = sess.run(tf.argmax(y_pred,1))
    #y_true = y_val
        
    #y_true = sess.run(tf.argmax(y_true,1))
    #print(len(y_pred),end=' ')
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))#, 'val = ','%f' %(accuracy_score(y_true, y_pred)) )
saver.save(sess, model_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]


Epoch: 0001 cost =  inf
Epoch: 0002 cost =  inf
Epoch: 0003 cost =  inf
Epoch: 0004 cost =  inf
Epoch: 0005 cost =  inf
Epoch: 0006 cost =  inf
Epoch: 0007 cost =  inf
Epoch: 0008 cost =  inf
Epoch: 0009 cost =  inf
Epoch: 0010 cost =  inf
Epoch: 0011 cost =  inf
Epoch: 0012 cost =  inf
Epoch: 0013 cost =  inf
Epoch: 0014 cost =  inf
Epoch: 0015 cost =  inf
Epoch: 0016 cost =  inf
Epoch: 0017 cost =  inf
Epoch: 0018 cost =  inf
Epoch: 0019 cost =  inf
Epoch: 0020 cost =  inf
Epoch: 0021 cost =  inf
Epoch: 0022 cost =  inf
Epoch: 0023 cost =  inf
Epoch: 0024 cost =  inf
Epoch: 0025 cost =  inf
Epoch: 0026 cost =  inf
Epoch: 0027 cost =  inf
Epoch: 0028 cost =  inf
Epoch: 0029 cost =  inf
Epoch: 0030 cost =  inf
Epoch: 0031 cost =  inf
Epoch: 0032 cost =  inf
Epoch: 0033 cost =  inf
Epoch: 0034 cost =  inf
Epoch: 0035 cost =  inf
Epoch: 0036 cost =  inf
Epoch: 0037 cost =  inf
Epoch: 0038 cost =  inf
Epoch: 0039 cost =  inf
Epoch: 0040 cost =  inf
Epoch: 0041 cost =  inf
Epoch: 0042 cost

Epoch: 0362 cost =  inf
Epoch: 0363 cost =  inf
Epoch: 0364 cost =  inf
Epoch: 0365 cost =  inf
Epoch: 0366 cost =  inf
Epoch: 0367 cost =  inf
Epoch: 0368 cost =  inf
Epoch: 0369 cost =  inf
Epoch: 0370 cost =  inf
Epoch: 0371 cost =  inf
Epoch: 0372 cost =  inf
Epoch: 0373 cost =  inf
Epoch: 0374 cost =  inf
Epoch: 0375 cost =  inf
Epoch: 0376 cost =  inf
Epoch: 0377 cost =  inf
Epoch: 0378 cost =  inf
Epoch: 0379 cost =  inf
Epoch: 0380 cost =  inf
Epoch: 0381 cost =  inf
Epoch: 0382 cost =  inf
Epoch: 0383 cost =  inf
Epoch: 0384 cost =  inf
Epoch: 0385 cost =  inf
Epoch: 0386 cost =  inf
Epoch: 0387 cost =  inf
Epoch: 0388 cost =  inf
Epoch: 0389 cost =  inf
Epoch: 0390 cost =  inf
Epoch: 0391 cost =  inf
Epoch: 0392 cost =  inf
Epoch: 0393 cost =  inf
Epoch: 0394 cost =  inf
Epoch: 0395 cost =  inf
Epoch: 0396 cost =  inf
Epoch: 0397 cost =  inf
Epoch: 0398 cost =  inf
Epoch: 0399 cost =  inf
Epoch: 0400 cost =  inf
Epoch: 0401 cost =  inf
Epoch: 0402 cost =  inf
Epoch: 0403 cost

'../../model/CNN/cnn_model'

## Prediction

In [25]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [26]:
# Print Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.913
Accuracy:  0.9130434782608695
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       0.00      0.00      0.00         2

   micro avg       0.91      0.91      0.91        23
   macro avg       0.46      0.50      0.48        23
weighted avg       0.83      0.91      0.87        23

[[21  0]
 [ 2  0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
